In [16]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession


conf = SparkConf().setAppName("test").set("spark.jars", "/opt/spark/jars/postgres-42.2.12.jar").\
                    set("spark.driver.extraClassPath", "/opt/spark/jars/postgres-42.2.12.jar").\
                    set("spark.executer.extraClassPath", "/opt/spark/jars/postgres-42.2.12.jar")

sc = SparkContext(conf=conf)




ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=test, master=local[*]) created by __init__ at <ipython-input-8-5faed4471bf6>:9 

In [21]:
import requests
import json
from pyspark.sql import Row
from collections import OrderedDict
from collections import OrderedDict
spark = SparkSession(sc)
URL = "https://api.covid19api.com/country/germany/status/confirmed/live?from=2020-03-01T00:00:00Z&to=2020-04-01T00:00:00Z"

r = requests.get(url =URL)
data = r.json()

def convert_to_row(d: dict) -> Row:
    return Row(**OrderedDict(sorted(d.items())))

df=sc.parallelize(data).map(convert_to_row).toDF()

jdbcDF=df.select("Cases", "Country","Date","Status")

jdbcDF.write.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/test") \
    .option("dbtable", "test.germanydata") \
    .option("user", "detian") \
    .option("password", "p31415926") \
    .save()
